# 02_03_numpycompute.ipynb -  Matrix operations with NumPy

In [ ]:
import math

import numpy as np
import matplotlib.pyplot as pp

**Poisson equation**:

$$\frac{\partial^2 \phi(x,y)}{\partial x^2} + \frac{\partial^2 \phi(x,y)}{\partial y^2} = 0$$

**Finite-difference representation of second derivatives:**

$$\frac{\partial^2 \phi(x_i,y_j)}{\partial x^2} + \frac{\partial^2 \phi(x_i,y_j)}{\partial y^2} \simeq
\phi(x_{i-1},y_j) + \phi(x_{i+1},y_j) + \phi(x_i,y_{j-1}) + \phi(x_i,y_{j+1}) - 4 \phi(x_i,y_j)$$

In [ ]:
n = 64
phi = np.zeros((n,n), 'd')  # 'd' is shorthand for np.float64

In [ ]:
phi[-1,:] = 1

In [ ]:
# show transpose and set origin='lower' for Cartesian convention  
pp.imshow(phi.T, origin='lower', extent=(0,1,0,1))

**Boundary conditions:**

$$\begin{eqnarray}\phi(x=0, y) &=& 0 \\
\phi(x, y=0) &=& 0 \\
\phi(x, y=1) &=& \sin(2 \pi x) \\
\phi(x=1, y) &=& -\sin(2 \pi y)\end{eqnarray}$$

In [ ]:
dx = 1/n
xs = np.linspace(0.5*dx, 1-0.5*dx, n)

In [ ]:
xs

In [ ]:
for i in range(n):
    phi[i,-1] = math.sin(2 * math.pi * xs[i])

In [ ]:
np.sin(2 * math.pi * xs)

In [ ]:
phi[:,-1] = np.sin(2 * math.pi * xs)

In [ ]:
phi[-1,:] = -np.sin(2 * math.pi * xs)

In [ ]:
def initphi(n=64):
    dx = 1/n
    xs = np.linspace(0.5*dx, 1-0.5*dx, n)
    
    phi = np.zeros((n,n), 'd')
    
    phi[:,-1] = np.sin(2 * math.pi * xs)
    phi[-1,:] = -np.sin(2 * math.pi * xs)
    
    return phi

In [ ]:
def showphi(array, colorbar=True):
    pp.imshow(array.T, origin='lower', extent=(0,1,0,1),
              vmin=-1, vmax=1, cmap='coolwarm')
    
    if colorbar:
        pp.colorbar()

In [ ]:
showphi(initphi())

In [ ]:
def jacobi(array):
    # make a copy of the array to store new values
    updated = array.copy()
    
    # iterate only on the "bulk", using but not changing boundary values 
    for i in range(1, n-1):
        for j in range(1, n-1):
            updated[i,j] = (array[i-1,j] + array[i+1,j] + array[i,j-1] + array[i,j+1]) / 4
            
    return updated

In [ ]:
showphi(jacobi(phi))

In [ ]:
%timeit jacobi(phi)

In [ ]:
phi[0:-2,1:-1] + phi[2:,1:-1]

In [ ]:
def matrixjacobi(array):
    updated = array.copy()
    
    updated[1:-1,1:-1] = (array[0:-2,1:-1] + array[2:,1:-1] + array[1:-1,0:-2] + array[1:-1,2:]) / 4
    
    return updated

In [ ]:
%timeit matrixjacobi(phi)

In [ ]:
def residual(array):
    laplace = array[0:-2,1:-1] + array[2:,1:-1] + array[1:-1,0:-2] + array[1:-1,2:] - 4*array[1:-1,1:-1]
    return np.mean(np.abs(laplace))

In [ ]:
pp.figure(figsize=(12,12))

phi = initphi(64)

# loop from 1 to 16 (subplot indexes from one)
for i in range(1, 1+16):
    pp.subplot(4, 4, i)
    showphi(phi, colorbar=False)
        
    for j in range(32):
        phi = matrixjacobi(phi)
    
    meanres = residual(phi)
    pp.title(f'Iteration {i*32}: {meanres:.2e}')
    
# save space among subplots
pp.tight_layout()